SET100 Data: Feature Set 01

Use class StockFS as container for each stock.  


Eikon data  
Price	MV	VA	VO	DY  

Join with Sector data  
Ticker	Sector	Industry  

Merge with YahooFinance  
High	Low	Open	Close	Volume	Adj Close



208164 rows × 26 columns   _fs1.csv
12M	_fs1.csv.gz

In [1]:
import sys, os, os.path
if not 'EXPERIMENT_HOME' in os.environ: # Configure local paths if not already defined
    ML_HOME = os.path.abspath(os.path.join("..", "ml_home")) # ML workspace
    EXPERIMENT_HOME = os.path.abspath(os.path.join(ML_HOME, "..")) # Experiment workspace
    DATA_HOME = os.path.abspath(os.path.join(EXPERIMENT_HOME,"dataset")) # Dataset location
    os.environ["EXPERIMENT_HOME"] = EXPERIMENT_HOME
    os.environ["DATA_HOME"] = DATA_HOME
    sys.path.insert(0, EXPERIMENT_HOME)
    sys.path.insert(0, ML_HOME) # Add to path so can load our library
    os.chdir(EXPERIMENT_HOME) # Change working directory to experiment workspace
print("Experiment Home: ", os.path.abspath(os.curdir), "; Data Home:", DATA_HOME)

Experiment Home:  /home/jovyan ; Data Home: /home/jovyan/dataset


In [2]:
from datetime import datetime
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
import xlrd
import matplotlib.pyplot as plt
from preprocessing import add_feature_set_1
%matplotlib inline


In [3]:
os.chdir("/home/jovyan/dataset/set100.data")
stocks = pd.read_csv('_directory.csv', index_col=0)
print("Stocks count:", len(stocks))
stocks.tail()

Stocks count: 150


,company_name,industry,sector,symbol
ticker,,,,
VIBHA,VIBHAVADI MEDICAL CENTER PUBLIC COMPANY LIMITED,Services,Health Care Services,VIBHA
VNG,VANACHAI GROUP PUBLIC COMPANY LIMITED,Property & Construction,Construction Materials,VNG
WHA,WHA CORPORATION PUBLIC COMPANY LIMITED,Property & Construction,Property Development,WHA
WHAUP,WHA UTILITIES AND POWER PUBLIC COMPANY LIMITED,Resources,Energy & Utilities,WHAUP
WORK,WORKPOINT ENTERTAINMENT PUBLIC COMPANY LIMITED,Services,Media & Publishing,WORK


In [4]:
period_start='2014-01-01'
period_end='2019-12-31'
series = {}
for s in stocks.symbol.values:
    df = pd.read_csv("%s.csv" % s, index_col=0)
    # Add column ticker
    df.loc[:,('Date')] = df.index
    df['Ticker'] = s
    
    # Add sector column
    sector = stocks[stocks['symbol']==s].sector.values[0]
    df['Sector'] = sector

    # Add industry column
    industry = stocks[stocks['symbol']==s].industry.values[0]
    df['Industry'] = industry
    
    # Add to dictionary
    series[s] = df[period_start:period_end]

In [5]:
# Select single stock series for analysis    
ticker = "SCB"
df = series[ticker]
df.index = pd.to_datetime(df.index)
df

,Price,MV,VA,VO,DY,Date,Ticker,Sector,Industry
Date,,,,,,,,,
2014-01-01,143.5,487040.3,NaN,NaN,3.48,2014-01-01,SCB,Banking,Financials
2014-01-02,131.5,446312.2,2001558.0,14727.9,3.80,2014-01-02,SCB,Banking,Financials
2014-01-03,132.5,449706.3,2051395.0,15411.3,3.77,2014-01-03,SCB,Banking,Financials
2014-01-06,140.0,475161.3,1740385.0,12798.9,3.57,2014-01-06,SCB,Banking,Financials
2014-01-07,144.0,488737.4,1322406.0,9353.2,3.47,2014-01-07,SCB,Banking,Financials
...,...,...,...,...,...,...,...,...,...
2019-12-25,119.5,405771.6,457841.1,3841.8,4.60,2019-12-25,SCB,Banking,Financials
2019-12-26,119.0,404073.9,630049.9,5295.4,4.62,2019-12-26,SCB,Banking,Financials
2019-12-27,119.5,405771.6,667488.9,5586.6,4.60,2019-12-27,SCB,Banking,Financials


In [6]:
# Stack dataframes together
# https://stackoverflow.com/questions/29351840/stack-two-pandas-data-frames
rows = []
for k, d in series.items():
    rows.append(d)
df = pd.concat(rows, ignore_index=True)
df

,Price,MV,VA,VO,DY,Date,Ticker,Sector,Industry
0,3.66,17751.00,NaN,NaN,0.00,2014-01-01,AAV,Transportation & Logistics,Services
1,3.44,16683.99,65373.3,19045.1,0.00,2014-01-02,AAV,Transportation & Logistics,Services
2,3.42,16586.99,34585.3,10163.8,0.00,2014-01-03,AAV,Transportation & Logistics,Services
3,3.54,17168.99,41467.2,11936.8,0.00,2014-01-06,AAV,Transportation & Logistics,Services
4,3.72,18041.99,92143.9,25572.0,0.00,2014-01-07,AAV,Transportation & Logistics,Services
...,...,...,...,...,...,...,...,...,...
227380,13.60,6005.22,25806.5,1918.2,3.31,2019-12-25,WORK,Media & Publishing,Services
227381,13.90,6137.69,26514.5,1907.0,3.24,2019-12-26,WORK,Media & Publishing,Services
227382,14.10,6226.00,22389.9,1587.2,3.19,2019-12-27,WORK,Media & Publishing,Services
227383,14.00,6181.84,2242.3,159.0,3.21,2019-12-30,WORK,Media & Publishing,Services


In [7]:
# Pivot table: Value=Price
ptable = df.pivot_table(values='Price', index='Date', columns=['Industry', 'Sector', 'Ticker'])
ptable

Industry   Agro & Food Industry                                            \
Sector             Agribusiness        Food & Beverage                      
Ticker                     GFPT    STA             CBG    CPF  ICHI  KTIS   
Date                                                                        
2014-01-01                 12.8  12.53             NaN  32.00   NaN   NaN   
2014-01-02                 12.2  11.95             NaN  30.50   NaN   NaN   
2014-01-03                 12.0  11.56             NaN  30.50   NaN   NaN   
2014-01-06                 11.9  11.47             NaN  29.25   NaN   NaN   
2014-01-07                 12.2  11.95             NaN  30.50   NaN   NaN   
...                         ...    ...             ...    ...   ...   ...   
2019-12-25                 11.9   9.80           82.75  27.75  4.82  4.16   
2019-12-26                 12.1   9.95           83.75  27.75  5.00  4.18   
2019-12-27                 12.4  10.10           84.50  27.50  4.92  4.16   
2019-12-30                 12.3  10.00           84.00  27.50  5.05  4.20   
2019-12-31                 12.3  10.00           84.00  27.50  5.05  4.20   

Industry                                ...            Technology  \
Sector                                  ... Electronic Components   
Ticker          M  MALEE   MINT    OSP  ...                   SVI   
Date                                    ...                         
2014-01-01  51.25  15.00  18.54    NaN  ...                  4.14   
2014-01-02  48.25  14.62  17.01    NaN  ...                  4.00   
2014-01-03  48.75  15.37  16.75    NaN  ...                  4.10   
2014-01-06  48.00  15.12  18.00    NaN  ...                  3.94   
2014-01-07  48.75  16.12  18.80    NaN  ...                  4.06   
...           ...    ...    ...    ...  ...                   ...   
2019-12-25  70.00   6.55  35.21  39.75  ...                  3.46   
2019-12-26  69.00   6.60  35.21  40.00  ...                  3.50   
2019-12-27  70.00   6.60  35.21  40.25  ...                  3.56   
2019-12-30  70.50   6.50  35.46  40.50  ...                  3.60   
2019-12-31  70.50   6.50  35.46  40.50  ...                  3.60   

Industry                                                                     \
Sector     Information & Communication Technology                             
Ticker                                     ADVANC   DTAC INTUCH   JAS JMART   
Date                                                                          
2014-01-01                                  199.5  97.00  67.75  6.04  8.61   
2014-01-02                                  187.0  93.00  64.00  5.59  8.18   
2014-01-03                                  196.0  92.50  65.50  5.41  7.92   
2014-01-06                                  201.0  95.25  66.00  5.72  7.92   
2014-01-07                                  210.0  96.25  67.75  5.90  7.80   
...                                           ...    ...    ...   ...   ...   
2019-12-25                                  215.0  53.25  57.50  5.00  8.05   
2019-12-26                                  215.0  53.75  57.50  4.98  8.30   
2019-12-27                                  214.0  53.25  57.25  4.98  8.25   
2019-12-30                                  213.0  53.25  57.25  5.00  8.35   
2019-12-31                                  213.0  53.25  57.25  5.00  8.35   

Industry                               
Sector                                 
Ticker     SAMART SAMTEL  THCOM  TRUE  
Date                                   
2014-01-01  14.80  12.90  40.50  6.51  
2014-01-02  14.00  12.10  38.00  5.90  
2014-01-03  14.10  11.70  36.75  5.82  
2014-01-06  14.90  12.00  36.25  5.99  
2014-01-07  14.80  12.40  36.75  6.17  
...           ...    ...    ...   ...  
2019-12-25   6.85   7.90   3.84  4.62  
2019-12-26   6.80   7.90   3.78  4.66  
2019-12-27   6.80   7.90   3.76  4.62  
2019-12-30   6.80   7.95   3.86  4.60  
2019-12-31   6.80   7.95   3.86  4.60  

[1565 rows x 150 colu

# Enrich each stock with data and features

In [8]:
class StockFS1:
    """Feature set 1"""
    def __init__(self, ticker):
        self.ticker = ticker
        df = series[ticker]
        df.index = pd.to_datetime(df.index)
        self.df = df
        
        self.outfile = "%s_FS1.csv" % self.ticker
        
        # YahooFinance
        self.merge_yahoofinance()
        
        # Add Feature Set 01
        add_feature_set_1(self.df)
        
    def merge_yahoofinance(self):
        symbol = self.ticker
        if symbol == 'GOLD': # Exception: change to FPT
            symbol = 'FPT'
        srcfile = os.path.join(DATA_HOME,"historical","%s.csv" % symbol)
        df_yahoo = pd.read_csv(srcfile)
        df_yahoo = df_yahoo.set_index('Date')
        df_yahoo.index = pd.to_datetime(df_yahoo.index)
        
        df_res = self.df.merge(df_yahoo, left_on=self.df.index, right_on=df_yahoo.index)
        df_res = df_res.set_index('key_0')
        df_res.index.rename('Date', inplace=True)
        self.df = df_res
        return True
    
    def save_features(self):
        self.df.to_csv(self.outfile)

In [9]:
fs1 = StockFS1('SCB')

# Manual step to merge with Yahoo Finance data series
ticker = 'SCB'
srcfile = os.path.join(DATA_HOME,"historical","%s.csv" % ticker)
df = pd.read_csv(srcfile)
df = df.set_index('Date')
df.index = pd.to_datetime(df.index)
df

/opt/conda/lib/python3.7/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/opt/conda/lib/python3.7/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-01-02,142.0,131.50,141.50,131.50,14726700.0,98.861626
2014-01-03,135.5,131.00,132.00,132.50,15410800.0,99.613434
2014-01-06,140.0,130.50,132.00,140.00,12798300.0,105.251930
2014-01-07,144.0,139.00,141.00,144.00,9352800.0,108.259125
2014-01-08,144.5,140.00,143.50,140.50,10750100.0,105.627823
...,...,...,...,...,...,...
2020-12-25,88.0,86.75,87.00,87.50,5857100.0,87.500000
2020-12-28,89.0,85.25,87.75,85.25,10926000.0,85.250000
2020-12-29,88.0,85.25,85.25,87.75,7755200.0,87.750000


In [10]:
# Don't run below
"""
df_res = fs1.df.merge(df, left_on=fs1.df.index, right_on=df.index)
df_res = df_res.set_index('key_0')
df_res.index.rename('Date', inplace=True)
"""

# df2 = pd.concat([fs1.df, df],sort=True)
# df2['2015-01-01':'2019-12-31']

"\ndf_res = fs1.df.merge(df, left_on=fs1.df.index, right_on=df.index)\ndf_res = df_res.set_index('key_0')\ndf_res.index.rename('Date', inplace=True)\n"

In [11]:
fs1 = StockFS1('BLAND')

In [12]:
fs1.df

,Price,MV,VA,VO,DY,Date,Ticker,Sector,Industry,High,...,FF-02,FF-03,FF-04,FF-05,FF-06,FF-07,FF-08,FF-09,FF-11,FF-12
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,1.40,28901.43,209862.1,144636.2,2.14,2014-01-02,BLAND,Property Development,Property & Construction,1.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03,1.38,28488.54,146089.1,105588.9,2.17,2014-01-03,BLAND,Property Development,Property & Construction,1.41,...,0.014389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-06,1.37,28282.12,118507.6,87387.1,2.19,2014-01-06,BLAND,Property Development,Property & Construction,1.38,...,0.007273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-07,1.42,29314.30,178610.4,127305.0,2.11,2014-01-07,BLAND,Property Development,Property & Construction,1.43,...,-0.035846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-08,1.42,29314.30,149637.8,104816.1,2.11,2014-01-08,BLAND,Property Development,Property & Construction,1.45,...,0.000000,1.398,1.403333,1.398000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,1.37,23802.95,3099.4,2254.9,6.57,2019-12-24,BLAND,Property Development,Property & Construction,1.38,...,0.007273,1.422,1.404667,1.398808,49.281555,0.05,25.641002,53.846135,-59.999988,NaN
2019-12-25,1.38,23976.69,7192.6,5211.6,6.52,2019-12-25,BLAND,Property Development,Property & Construction,1.39,...,-0.007273,1.408,1.390667,1.392539,51.157618,0.05,12.820501,30.769213,-54.999994,NaN
2019-12-26,1.36,23629.20,14437.9,10576.7,6.62,2019-12-26,BLAND,Property Development,Property & Construction,1.38,...,0.014599,1.384,1.374667,1.381692,47.400477,-0.02,10.073251,16.178251,-64.999982,NaN


In [13]:
# add_feature_set_1(fs1.df)

In [14]:
fs1.df.columns

Index(['Price', 'MV', 'VA', 'VO', 'DY', 'Date', 'Ticker', 'Sector', 'Industry',
       'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close', 'FF-01', 'FF-02',
       'FF-03', 'FF-04', 'FF-05', 'FF-06', 'FF-07', 'FF-08', 'FF-09', 'FF-11',
       'FF-12'],
      dtype='object')

In [15]:
fs1.df.columns.values

array(['Price', 'MV', 'VA', 'VO', 'DY', 'Date', 'Ticker', 'Sector',
       'Industry', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close',
       'FF-01', 'FF-02', 'FF-03', 'FF-04', 'FF-05', 'FF-06', 'FF-07',
       'FF-08', 'FF-09', 'FF-11', 'FF-12'], dtype=object)

# Compute features and save

In [16]:
stocks = []
for t in series.keys():
    s1 = StockFS1(t)
    s1.save_features()
    print("Saved %s." % t)
    stocks.append(s1.df)

Saved AAV.
Saved ADVANC.
Saved AEONTS.
Saved AMATA.
Saved ANAN.
Saved AOT.
Saved AP.
Saved ASP.
Saved AWC.
Saved BA.
Saved BANPU.
Saved BAY.
Saved BBL.
Saved BCH.
Saved BCP.
Saved BCPG.
Saved BDMS.
Saved BEAUTY.
Saved BEC.
Saved BEM.
Saved BGRIM.
Saved BH.


/opt/conda/lib/python3.7/site-packages/ta/trend.py:798: RuntimeWarning: invalid value encountered in double_scalars
  dip[i + self._window] = 100 * (self._dip[i] / self._trs[i])
/opt/conda/lib/python3.7/site-packages/ta/trend.py:813: RuntimeWarning: invalid value encountered in double_scalars
  din[i + self._window] = 100 * (self._din[i] / self._trs[i])


Saved BIG.
Saved BJC.
Saved BJCHI.
Saved BLA.
Saved BLAND.
Saved BPP.
Saved BTS.
Saved CBG.
Saved CENTEL.
Saved CHG.
Saved CK.
Saved CKP.
Saved COM7.
Saved CPALL.
Saved CPF.
Saved CPN.
Saved DELTA.
Saved DEMCO.
Saved DTAC.
Saved EA.
Saved EGCO.
Saved EPG.
Saved ERW.
Saved ESSO.
Saved GOLD.
Saved GFPT.
Saved GGC.
Saved GL.
Saved GLOBAL.
Saved GPSC.
Saved GULF.
Saved GUNKUL.
Saved HANA.
Saved HMPRO.
Saved ICHI.
Saved INTUCH.
Saved IRPC.
Saved ITD.
Saved IVL.
Saved JAS.
Saved JMART.
Saved JMT.
Saved JWD.
Saved KAMART.
Saved KBANK.
Saved KCE.
Saved KKP.
Saved KTB.
Saved KTC.
Saved KTIS.
Saved LH.
Saved LHFG.
Saved LOXLEY.
Saved LPN.
Saved M.
Saved MAJOR.
Saved MALEE.
Saved MBK.
Saved MC.
Saved MEGA.
Saved MINT.
Saved MONO.
Saved MTC.
Saved NOK.
Saved ORI.
Saved OSP.
Saved PLANB.
Saved PLAT.
Saved PRM.
Saved PSH.
Saved PSL.
Saved PTG.
Saved PTL.
Saved PTT.
Saved PTTEP.
Saved PTTGC.
Saved QH.
Saved RATCH.
Saved RS.
Saved S.
Saved SAMART.
Saved SAMTEL.
Saved SAPPE.
Saved SAWAD.
Saved SCB.
Sav

In [19]:
# Stack dataframes together
# https://stackoverflow.com/questions/29351840/stack-two-pandas-data-frames

df_all = pd.concat(stocks, ignore_index=True)
df_all

,Price,MV,VA,VO,DY,Date,Ticker,Sector,Industry,High,...,FF-02,FF-03,FF-04,FF-05,FF-06,FF-07,FF-08,FF-09,FF-11,FF-12
0,3.44,16683.99,65373.3,19045.1,0.00,2014-01-02,AAV,Transportation & Logistics,Services,3.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.42,16586.99,34585.3,10163.8,0.00,2014-01-03,AAV,Transportation & Logistics,Services,3.44,...,0.005831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.54,17168.99,41467.2,11936.8,0.00,2014-01-06,AAV,Transportation & Logistics,Services,3.54,...,-0.034486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.72,18041.99,92143.9,25572.0,0.00,2014-01-07,AAV,Transportation & Logistics,Services,3.72,...,-0.049597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.64,17653.99,57568.1,15618.6,0.00,2014-01-08,AAV,Transportation & Logistics,Services,3.76,...,0.021740,3.552,3.598667,3.552000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208159,13.40,5916.91,43853.1,3283.1,3.36,2019-12-24,WORK,Media & Publishing,Services,13.70,...,0.014815,13.640,13.593333,13.571773,32.716036,0.2,43.668827,45.310253,-69.230803,NaN
208160,13.60,6005.22,25806.5,1918.2,3.31,2019-12-25,WORK,Media & Publishing,Services,13.80,...,-0.014815,13.620,13.580000,13.581182,35.937565,0.5,42.803025,43.506496,-53.846131,NaN
208161,13.90,6137.69,26514.5,1907.0,3.24,2019-12-26,WORK,Media & Publishing,Services,14.10,...,-0.021819,13.660,13.673333,13.687455,40.514878,0.5,57.575726,48.015859,-30.769270,NaN
208162,14.10,6226.00,22389.9,1587.2,3.19,2019-12-27,WORK,Media & Publishing,Services,14.30,...,-0.014286,13.720,13.820000,13.824970,43.403378,0.3,73.659664,58.012805,-15.384598,NaN


In [21]:
df_all.to_csv('_fs1.csv.gz')